In [2]:
from polygon import RESTClient

# Replace 'your_api_key' with your actual Polygon.io API key
api_key = "UVu97Jw0QqDcmCbK1EGwjXP2I9nhu3cJ"
client = RESTClient(api_key)

# Define parameters
ticker = "SPY"
expiration_date = "2025-01-08"  # Expiration date in YYYY-MM-DD format

# Fetch the option chain
options = client.list_options_contracts(
    underlying_ticker=ticker,
    #expiration_date=expiration_date
)

# Fetch the underlying price
#underlying_quote = client.get_last_quote(ticker)
#underlying_price = (underlying_quote.ask + underlying_quote.bid) / 2  # Midpoint price
underlying_price = 590
# Print the results
print(f"Underlying SPY Price: {underlying_price:.2f}")
print("\nOptions Chain:")
#for option in options:
    #print(f"Symbol: {option.ticker}, Expiration: {option.expiration_date}, Strike: {option.strike_price}")

Underlying SPY Price: 590.00

Options Chain:


In [3]:
from polygon import RESTClient
import datetime

# Replace with your Polygon.io API key
api_key = "your_api_key"
client = RESTClient(api_key)

def fetch_underlying_price(client, ticker, date):
    """Fetch the closing price of the underlying ticker for a given date."""
    try:
        aggs = client.get_aggs(
            ticker=ticker,
            multiplier=1,
            timespan="day",
            from_=date,
            to=date
        )
        if aggs and len(aggs) > 0:
            return aggs[0].close  # Close price of the underlying
        else:
            print(f"No underlying data for {ticker} on {date}")
            return None
    except Exception as e:
        print(f"Error fetching underlying price for {ticker} on {date}: {e}")
        return None

def generate_option_tickers(underlying, expiration, close_price, strike_inc, strike_range):
    """Generate option tickers for a given close price, expiration, and strike range."""
    strikes = range(
        int(close_price - strike_range),
        int(close_price + strike_range) + strike_inc,
        strike_inc
    )
    call_symbols = [f"O:{underlying}{expiration}C{strike * 1000:08d}" for strike in strikes]
    put_symbols = [f"O:{underlying}{expiration}P{strike * 1000:08d}" for strike in strikes]
    return call_symbols + put_symbols

def fetch_option_quotes(client, tickers, date):
    """Fetch all quotes for a list of option tickers."""
    all_quotes = []
    for ticker in tickers:
        try:
            # Use `get_quotes` to fetch all quotes for the ticker
            quotes = client.list_quotes(
                ticker=ticker,
                limit=5000,  # Adjust limit as necessary
                timestamp=date
            )
            for quote in quotes:
                all_quotes.append({
                    "ticker": ticker,
                    "bid_price": quote.bid_price,
                    "ask_price": quote.ask_price,
                    "timestamp": quote.timestamp
                })
        except Exception as e:
            print(f"Error fetching quotes for {ticker}: {e}")
    return all_quotes

# Parameters
underlying_ticker = "SPY"
expiration = "250108"  # Expiration date (YYMMDD format)
day_range = ["2025-01-01", "2025-01-08"]  # Date range for analysis
strike_inc = 5  # Strike increment in dollars
strike_range = 50  # Strike range around the underlying price in dollars

# Main Script
all_quotes = []

for day in day_range:
    print(f"Processing data for {day}...")
    close_price = fetch_underlying_price(client, underlying_ticker, day)
    
    if close_price is not None:
        # Generate option tickers
        option_tickers = generate_option_tickers(
            underlying=underlying_ticker,
            expiration=expiration,
            close_price=close_price,
            strike_inc=strike_inc,
            strike_range=strike_range
        )
        
        # Fetch all quotes for the generated tickers
        quotes = fetch_option_quotes(client, option_tickers, day)
        all_quotes.extend(quotes)

        print(f"Fetched {len(quotes)} quotes for {day}.")

# Print all fetched quotes
for quote in all_quotes:
    print(f"Ticker: {quote['ticker']}, Bid: {quote['bid_price']}, Ask: {quote['ask_price']}, Time: {quote['timestamp']}")

print(f"Total quotes fetched: {len(all_quotes)}")

Processing data for 2025-01-01...
Error fetching underlying price for SPY on 2025-01-01: {"status":"ERROR","request_id":"8c773051e5dce5ad97c71d24332912fe","error":"Unknown API Key"}
Processing data for 2025-01-08...
Error fetching underlying price for SPY on 2025-01-08: {"status":"ERROR","request_id":"66e792df4f013ece47f86df8dbd96e7a","error":"Unknown API Key"}
Total quotes fetched: 0


In [5]:
def create_polygon_symbol(ticker, maturity, strike, option_type):
    """
    Create a Polygon symbol for an option.

    Parameters:
    ticker (str): The ticker symbol of the underlying asset.
    maturity (str): The expiration date of the option in YYYY-MM-DD format.
    strike (float): The strike price of the option.
    option_type (str): The type of the option ('C' for call, 'P' for put).

    Returns:
    str: The Polygon symbol for the option.
    """
    # Convert maturity date to YYMMDD format
    maturity_formatted = maturity[2:].replace("-", "")
    
    # Format the strike price to 8 digits with leading zeros
    strike_formatted = f"{int(strike * 1000):08d}"
    
    # Create the Polygon symbol
    polygon_symbol = f"O:{ticker}{maturity_formatted}{option_type}{strike_formatted}"
    
    return polygon_symbol

# Example usage
ticker = "SPY"
maturity = "2025-01-08"
strike = 610
option_type = "P"  # 'C' for call, 'P' for put
polygon_symbol = create_polygon_symbol(ticker, maturity, strike, option_type)
print(polygon_symbol)

O:SPY250108P00610000


In [7]:
from polygon import RESTClient
import datetime

# Replace with your Polygon.io API key
api_key = "your_api_key"
api_key = "UVu97Jw0QqDcmCbK1EGwjXP2I9nhu3cJ"
client = RESTClient(api_key)

def fetch_underlying_price(client, ticker, date):
    """Fetch the closing price of the underlying ticker for a given date."""
    try:
        aggs = client.get_aggs(
            ticker=ticker,
            multiplier=1,
            timespan="day",
            from_=date,
            to=date
        )
        if aggs and len(aggs) > 0:
            return aggs[0].close  # Close price of the underlying
        else:
            print(f"No underlying data for {ticker} on {date}")
            return None
    except Exception as e:
        print(f"Error fetching underlying price for {ticker} on {date}: {e}")
        return None

def generate_option_tickers(underlying, expiration, close_price, strike_inc, strike_range):
    """Generate option tickers for a given close price, expiration, and strike range."""
    strikes = range(
        int(close_price - strike_range),
        int(close_price + strike_range) + strike_inc,
        strike_inc
    )
    call_symbols = [f"O:{underlying}{expiration}C{strike * 1000:08d}" for strike in strikes]
    put_symbols = [f"O:{underlying}{expiration}P{strike * 1000:08d}" for strike in strikes]
    return call_symbols + put_symbols

def fetch_option_quotes(client, tickers, date):
    """Fetch all quotes for a list of option tickers."""
    all_quotes = []
    for ticker in tickers:
        try:
            # Use `list_quotes` to fetch all quotes for the ticker
            quotes = client.list_quotes(
                ticker=ticker,
                limit=5000,  # Adjust limit as necessary
                timestamp=date
            )
            for quote in quotes:
                all_quotes.append({
                    "ticker": ticker,
                    "bid_price": quote.bid_price,
                    "ask_price": quote.ask_price,
                    "bid_size": quote.bid_size,
                    "ask_size": quote.ask_size,
                    "bid_exchange": quote.bid_exchange,
                    "ask_exchange": quote.ask_exchange,
                    "sequence_number": quote.sequence_number,
                    "timestamp": quote.sip_timestamp
                })
        except Exception as e:
            print(f"Error fetching quotes for {ticker}: {e}")
    return all_quotes

# Parameters
underlying_ticker = "SPY"
expiration = "250108"  # Expiration date (YYMMDD format)
day_range = ["2025-01-01", "2025-01-08"]  # Date range for analysis
strike_inc = 5  # Strike increment in dollars
strike_range = 50  # Strike range around the underlying price in dollars

# Main Script
all_quotes = []

for day in day_range:
    print(f"Processing data for {day}...")
    close_price = fetch_underlying_price(client, underlying_ticker, day)
    
    if close_price is not None:
        print(f"Underlying close price on {day}: {close_price}")
        
        # Generate option tickers
        option_tickers = generate_option_tickers(
            underlying=underlying_ticker,
            expiration=expiration,
            close_price=close_price,
            strike_inc=strike_inc,
            strike_range=strike_range
        )
        
        # Fetch all quotes for the generated tickers
        quotes = fetch_option_quotes(client, option_tickers, day)
        all_quotes.extend(quotes)

        print(f"Fetched {len(quotes)} quotes for {day}.")

# Print all fetched quotes
#for quote in all_quotes:
    #print(f"Ticker: {quote['ticker']}, Bid: {quote['bid_price']}, Ask: {quote['ask_price']}, Time: {quote['timestamp']}")

print(f"Total quotes fetched: {len(all_quotes)}")

Processing data for 2025-01-01...
No underlying data for SPY on 2025-01-01
Processing data for 2025-01-08...
Underlying close price on 2025-01-08: 589.49
Fetched 1268533 quotes for 2025-01-08.
Total quotes fetched: 1268533


In [8]:
quotes
import pandas as pd
pd.DataFrame(quotes)
pd.DataFrame(quotes).to_csv("quotes.csv", index=False)


In [9]:
from datetime import datetime, timezone

def convert_sip_timestamp_to_datetime(sip_timestamp):
    """
    Convert SIP Unix Timestamp (nanoseconds) to a human-readable datetime.
    
    Parameters:
        sip_timestamp (int): SIP Unix Timestamp in nanoseconds.
    
    Returns:
        datetime: Datetime object in UTC.
    """
    # Convert nanoseconds to seconds
    seconds = sip_timestamp / 1_000_000_000
    # Convert to a datetime object
    return datetime.fromtimestamp(seconds, tz=timezone.utc)

# Example SIP Unix Timestamp
sip_timestamp = 1672617600000000000  # Replace with your SIP timestamp
converted_datetime = convert_sip_timestamp_to_datetime(sip_timestamp)

print(f"SIP Timestamp: {sip_timestamp}")
print(f"Converted Datetime: {converted_datetime}")

SIP Timestamp: 1672617600000000000
Converted Datetime: 2023-01-02 00:00:00+00:00


In [10]:
df=pd.DataFrame(quotes)

In [11]:
dt_times = df['timestamp'].apply(convert_sip_timestamp_to_datetime)
df['timestamp2'] = dt_times
max(df['timestamp2'])
min(df['timestamp2'])

Timestamp('2025-01-08 14:30:00.003617+0000', tz='UTC')

In [13]:
import pytz
eastern_tz = pytz.timezone("US/Eastern")
# Convert UTC to US Eastern Time
df_times2= df['timestamp2'].apply(lambda x: x.astimezone(eastern_tz))
df['timestamp3'] = df_times2
min(df['timestamp3'])
df.to_csv("quotes.csv", index=False)